In [1]:

import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model, sample_gapsplit, plot_grid_95_reactions
from load_modify_sample_utils import sample_dingo, sample_optgp, sampling_statistics
from correlations_utils import correlated_reactions, split_forward_reverse
from graphs_utils import construct_graph, plot_graph, compare_betweenness_centralities

/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/touliopoulos/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)

['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.0), 'D_LACt2': (-1000.0, 1000.0), 'ENO': (-1000.0, 1000.0), 'SUCCt3': (0.0, 1000.0), 'ETOHt2r': (-1000.0, 1000.0), 'SUCDi': (0.0, 10

In [ ]:

def get_kegg_identifiers(model):
    return (
        {met.id: met.annotation.get("kegg.compound") for met in model.metabolites},
        {reac.id: reac.annotation.get("kegg.reaction") for reac in model.reactions},
    )


dictionary = get_kegg_identifiers(ec_cobra_model)
print(dictionary[1])



{'1HIBUPGLU_Sabc': None, '1HIBUP_S_GLCAASE': None, '1HIBUP_Ste': None, '1HMDGLUC_GLCAASE': None, '1HMDGLUCabc': None, '1OHMDZte': None, '23DHMPO': None, '23PDE2': None, '23PDE4': None, '23PDE7': None, '23PDE9': None, '26DAPLLAT': None, '2AHBUTI': None, '2HATVACIDGLUC_GLCAASE': None, '2HATVACIDGLUCabc': None, '2HATVACIDte': None, '2HATVLACGLUC_GLCAASE': None, '2HATVLACGLUCabc': None, '2HATVLACte': None, '2HIBUPGLU_Sabc': None, '2HIBUP_S_GLCAASE': None, '2HIBUP_Ste': None, '2IMZS': None, '2MBCOATA': None, '2OH_CBZ_GLC_GLCAASE': None, '2OH_CBZ_GLCabc': None, '2OH_CBZte': None, '2OH_MTZ_GLC_GLCAASE': None, '2OH_MTZ_GLCabc': None, '2OH_MTZte': None, '3HAD100': None, '3HAD10M11': None, '3HAD10M12': None, '3HAD11M12': None, '3HAD120': None, '3HAD12M13': None, '3HAD12M14': None, '3HAD13M14': None, '3HAD140': None, '3HAD14M15': None, '3HAD14M16': None, '3HAD15M16': None, '3HAD160': None, '3HAD180': None, '3HAD40': None, '3HAD4M5': None, '3HAD4M6': None, '3HAD5M6': None, '3HAD60': None, '3HAD6M7

In [5]:

from bioservices import KEGG

kegg = KEGG()
model_to_kegg = dictionary[1]

reaction_to_pathways = {}

for model_rxn, kegg_rxn in model_to_kegg.items():
    if kegg_rxn:
        try:
            entry = kegg.get(kegg_rxn)
            parsed = kegg.parse(entry)
            pathways = parsed.get('PATHWAY', {})
            reaction_to_pathways[model_rxn] = list(pathways.values())
        except Exception as e:
            print(f"Error retrieving {kegg_rxn}: {e}")
            reaction_to_pathways[model_rxn] = []

# Print or save the mappings
import pandas as pd
df = pd.DataFrame([
    {'model_reaction': rxn, 'kegg_reaction': model_to_kegg[rxn], 'pathways': paths}
    for rxn, paths in reaction_to_pathways.items()
])


In [ ]:

df.head()


,model_reaction,kegg_reaction,pathways
0,PFL,R00212,"[Pyruvate metabolism, Butanoate metabolism, Me..."
1,PGK,R01512,"[Glycolysis / Gluconeogenesis, Carbon fixation..."
2,PGL,R02035,"[Pentose phosphate pathway, Metabolic pathways..."
3,ACALD,R00228,"[Phenylalanine metabolism, Benzoate degradatio..."
4,PGM,R01518,"[Glycolysis / Gluconeogenesis, Glycine, serine..."
